In [39]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('/Users/johannessolibieda/Documents/GitHub/Its-Wekk/4 - Data/4 - Johannes/100048.csv', delimiter=';')

# Display the first few rows of the DataFrame
df.head()
# Sort the DataFrame by the 'Datum/Zeit' column and reset the index
df = df.sort_values(by='Datum/Zeit').reset_index(drop=True)

# Display the first few rows of the DataFrame
df.head()

# Drop the first row of the DataFrame
df = df.drop(index=0).reset_index(drop=True)

# Display the first few rows of the DataFrame after dropping the first row
df.head()

,Datum/Zeit,timestamp_text,o3_stundenmittelwerte_ug_m3,geo_point_2d
0,2022-01-01T00:00:00+00:00,2022-01-01T01:00:00+00:00,54.712,"47.5717, 7.6870833"
1,2022-01-01T01:00:00+00:00,2022-01-01T02:00:00+00:00,55.592,"47.5717, 7.6870833"
2,2022-01-01T02:00:00+00:00,2022-01-01T03:00:00+00:00,52.945,"47.5717, 7.6870833"
3,2022-01-01T03:00:00+00:00,2022-01-01T04:00:00+00:00,52.387,"47.5717, 7.6870833"
4,2022-01-01T04:00:00+00:00,2022-01-01T05:00:00+00:00,51.119,"47.5717, 7.6870833"


In [40]:
# Print the column names to find the correct column name
print(df.columns)

# Check for missing or empty values in the correct column
correct_column_name = 'o3_stundenmittelwerte_ug_m3'  # Replace with the actual column name
missing_values = df[correct_column_name].isnull().sum()
print(f"Number of missing values in '{correct_column_name}': {missing_values}")

# Check for zero values in the correct column
zero_values = (df[correct_column_name] == 0).sum()
print(f"Number of zero values in '{correct_column_name}': {zero_values}")

Index(['Datum/Zeit', 'timestamp_text', 'o3_stundenmittelwerte_ug_m3',
       'geo_point_2d'],
      dtype='object')
Number of missing values in 'o3_stundenmittelwerte_ug_m3': 0
Number of zero values in 'o3_stundenmittelwerte_ug_m3': 0


In [41]:
# Drop the specified columns
df = df.drop(columns=['geo_point_2d', 'timestamp_text'])

# Display the first few rows of the DataFrame after dropping the columns
df.head()

,Datum/Zeit,o3_stundenmittelwerte_ug_m3
0,2022-01-01T00:00:00+00:00,54.712
1,2022-01-01T01:00:00+00:00,55.592
2,2022-01-01T02:00:00+00:00,52.945
3,2022-01-01T03:00:00+00:00,52.387
4,2022-01-01T04:00:00+00:00,51.119


In [42]:
# Convert the 'Datum/Zeit' column to datetime
df['Datum/Zeit'] = pd.to_datetime(df['Datum/Zeit'])

# Create a date range with the expected frequency (hourly in this case)
expected_dates = pd.date_range(start=df['Datum/Zeit'].min(), end=df['Datum/Zeit'].max(), freq='H')

# Find the missing dates by comparing the expected dates with the actual dates
missing_dates = expected_dates.difference(df['Datum/Zeit'])

# Print the number of missing dates and the missing dates themselves
print(f"Number of missing dates: {len(missing_dates)}")
print("Missing dates:")
print(missing_dates)

Number of missing dates: 385
Missing dates:
DatetimeIndex(['2022-02-17 15:00:00+00:00', '2022-02-17 16:00:00+00:00',
               '2022-03-27 01:00:00+00:00', '2022-03-27 02:00:00+00:00',
               '2022-09-26 08:00:00+00:00', '2022-10-30 01:00:00+00:00',
               '2023-03-26 01:00:00+00:00', '2023-03-26 02:00:00+00:00',
               '2023-07-04 13:00:00+00:00', '2023-07-04 14:00:00+00:00',
               ...
               '2024-03-14 05:00:00+00:00', '2024-03-14 06:00:00+00:00',
               '2024-03-14 07:00:00+00:00', '2024-03-14 08:00:00+00:00',
               '2024-03-14 09:00:00+00:00', '2024-03-14 16:00:00+00:00',
               '2024-03-31 01:00:00+00:00', '2024-03-31 02:00:00+00:00',
               '2024-09-03 09:00:00+00:00', '2024-10-27 01:00:00+00:00'],
              dtype='datetime64[ns, UTC]', length=385, freq=None)


/var/folders/rt/8pf1f2_x7sj69ytpkljsggxr0000gn/T/ipykernel_48450/3482865412.py:5: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  expected_dates = pd.date_range(start=df['Datum/Zeit'].min(), end=df['Datum/Zeit'].max(), freq='H')


In [43]:
# Create a new DataFrame to store the filled values
filled_df = df.copy()

# Iterate over the missing dates
for missing_date in missing_dates:
    # Find the same time on the same weekday in the existing data
    same_time_weekday = df[(df['Datum/Zeit'].dt.hour == missing_date.hour) & (df['Datum/Zeit'].dt.weekday == missing_date.weekday())]
    
    # Calculate the mean value for the same time on the same weekday
    mean_value = same_time_weekday[correct_column_name].mean()
    
    # Create a new row with the missing date and the calculated mean value
    new_row = pd.DataFrame({'Datum/Zeit': [missing_date], correct_column_name: [mean_value]})
    
    # Append the new row to the filled DataFrame
    filled_df = pd.concat([filled_df, new_row], ignore_index=True)

# Sort the filled DataFrame by the 'Datum/Zeit' column and reset the index
filled_df = filled_df.sort_values(by='Datum/Zeit').reset_index(drop=True)

# Display the first few rows of the filled DataFrame
filled_df.head()

,Datum/Zeit,o3_stundenmittelwerte_ug_m3
0,2022-01-01 00:00:00+00:00,54.712
1,2022-01-01 01:00:00+00:00,55.592
2,2022-01-01 02:00:00+00:00,52.945
3,2022-01-01 03:00:00+00:00,52.387
4,2022-01-01 04:00:00+00:00,51.119


In [44]:
# Create a date range with the expected frequency (hourly in this case)
expected_dates = pd.date_range(start=filled_df['Datum/Zeit'].min(), end=filled_df['Datum/Zeit'].max(), freq='H')

# Find the missing dates by comparing the expected dates with the actual dates in filled_df
missing_dates = expected_dates.difference(filled_df['Datum/Zeit'])

# Print the number of missing dates and the missing dates themselves
print(f"Number of missing dates: {len(missing_dates)}")
print("Missing dates:")
print(missing_dates)

Number of missing dates: 0
Missing dates:
DatetimeIndex([], dtype='datetime64[ns, UTC]', freq='h')


/var/folders/rt/8pf1f2_x7sj69ytpkljsggxr0000gn/T/ipykernel_48450/3248014503.py:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  expected_dates = pd.date_range(start=filled_df['Datum/Zeit'].min(), end=filled_df['Datum/Zeit'].max(), freq='H')


In [45]:
filled_df.to_csv('1000048cleaned.csv', index=False)